In [1]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import os

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER'] = 'uploads/'  # Folder where uploaded images will be stored
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max-upload size limit

# Ensure the upload folder exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/')
def index():
    return render_template('/content/home.html')  # This will be your upload form

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        # Perform OCR on the file
        model = ocr_predictor(pretrained=True)
        img = DocumentFile.from_images(file_path)
        result = model(img)
        output = result.export()

        # Search for the string
        target_string = "Muhammad Danish Bin Zainudin"  # Replace with the string you want to search for
        full_text = " ".join([obj3["value"] for obj1 in output['pages'][0]["blocks"] for obj2 in obj1["lines"] for obj3 in obj2["words"]])

        if target_string.lower() in full_text.lower():
            return f"The string '{target_string}' was found in the document."
        else:
            return f"The string '{target_string}' was not found in the document."

    return 'File Uploaded Successfully'

if __name__ == '__main__':
    app.run()


C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\weasyprint\text\fonts.py:62: UserWarning: FontConfig cannot load default config file. Expect ugly output.
  warn('FontConfig cannot load default config file. Expect ugly output.')


ModuleNotFoundError: No module named 'onnx.defs'

In [2]:
!pip install python-doctr

Defaulting to user installation because normal site-packages is not writeable
